In [90]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

import numpy as np
b = np.load("b.npy")

height = 64
spatial_norm = 32
b_norm = 100

In [91]:
b_bottom = b[:, :, 0, :]

Nx, Ny, _ = b_bottom.shape
Nz = height

cube_shape = (Nx, Ny, Nz)

def coords(xbounds, ybounds, zbounds):
    return np.stack(np.mgrid[xbounds[0]:xbounds[1]+1, ybounds[0]:ybounds[1]+1, zbounds[0]:zbounds[1]+1], axis=-1)

bottom_coords = coords((0, Nx-1), (0, Ny-1), (0, 0)).reshape(-1, 3)
bottom_values = b_bottom.reshape(-1, 3)

In [92]:
bottom_coords

array([[ 0,  0,  0],
       [ 0,  1,  0],
       [ 0,  2,  0],
       ...,
       [63, 61,  0],
       [63, 62,  0],
       [63, 63,  0]])

In [93]:
bottom_values

array([[ 0.1655598 ,  1.18534835, -2.10841989],
       [ 0.28285207,  1.30286912, -2.19860577],
       [ 0.41426848,  1.42453679, -2.28830561],
       ...,
       [-0.41426914, -1.42453732, -2.28830531],
       [-0.28285279, -1.30286958, -2.1986055 ],
       [-0.1655598 , -1.18534835, -2.10841989]])

In [94]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

top_lateral_coords = [coords((0, Nx-1), (0, Ny-1), (Nz-2, Nz)),
                      coords((-1, 1), (0, Ny-1), (0, Nz-1)),
                      coords((Nx-2, Nx), (0, Ny-1), (0, Nz-1)),
                      coords((0, Nx-1), (-1, 1), (0, Nz-1)),
                      coords((0, Nx-1), (Ny-2, Ny), (0, Nz-1))]

r_top_lateral_shape = [c.shape[:-1] for c in top_lateral_coords]
r_top_lateral = np.concatenate([c.reshape(((-1, 3))) for c in top_lateral_coords])
r_bottom = bottom_coords
bz_bottom = bottom_values[:, 2]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.array([[0, 0, 1/np.sqrt(2*np.pi)]])
        c = torch.tensor(c, dtype=torch.float32)
        self.register_buffer('c', c)

    def forward(self, r):
        numerator = self.b_n[:, None]
        denominator = torch.sqrt(torch.sum((r[None, :] - self.r_p[:, None] + self.c[None])**2, -1))
        potential = torch.sum(numerator/denominator, 0) / (2*np.pi)
        return potential
    
pf_batch_size = r_top_lateral.shape[0] // 100
with torch.no_grad():
    b_n = torch.tensor(bz_bottom, dtype=torch.float32)
    r_p = torch.tensor(r_bottom, dtype=torch.float32)
    model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

    r_coords= torch.tensor(r_top_lateral, dtype=torch.float32)

    potential = []
    for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
                        desc='Potential Boundary'):
        r = r.to(device)
        p_batch = model(r)
        potential += [p_batch.cpu()]

potential = torch.cat(potential).numpy()

idx = 0
fields = []
for s in r_top_lateral_shape:
    p = potential[idx:idx + np.prod(s)].reshape(s)
    b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
    fields += [b]
    idx += np.prod(s)

pf_fields = [fields[0][:, :, 1].reshape((-1, 3)),
            fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
            fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
pf_coords = [top_lateral_coords[0][:, :, 1].reshape((-1, 3)),
            top_lateral_coords[1][:, 1, :].reshape((-1, 3)), top_lateral_coords[2][:, 1, :].reshape((-1, 3)),
            top_lateral_coords[3][1, :, :].reshape((-1, 3)), top_lateral_coords[4][1, :, :].reshape((-1, 3))]

Potential Boundary: 100%|██████████| 101/101 [00:00<00:00, 183.33it/s]


In [95]:
pf_fields[0]

array([[-0.07973576, -0.04400277,  0.05472827],
       [-0.08081293, -0.04405975,  0.05578446],
       [-0.08187222, -0.04410911,  0.05687547],
       ...,
       [ 0.08186817,  0.04411054,  0.05687499],
       [ 0.0808115 ,  0.04406047,  0.05578375],
       [ 0.07973099,  0.04399967,  0.05472779]], dtype=float32)

In [96]:
pf_coords[0]

array([[ 0,  0, 63],
       [ 0,  1, 63],
       [ 0,  2, 63],
       ...,
       [63, 61, 63],
       [63, 62, 63],
       [63, 63, 63]])

In [97]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.array([[0, 0, 1/np.sqrt(2*np.pi)]])
        c = torch.tensor(c, dtype=torch.float32)
        self.register_buffer('c', c)

    def forward(self, r):
        numerator = self.b_n[:, None]
        denominator = torch.sqrt(torch.sum((r[None, :] - self.r_p[:, None] + self.c[None])**2, -1))
        potential = torch.sum(numerator/denominator, 0) / (2*np.pi)
        return potential


top_lateral_coords = [coords((0, Nx-1), (0, Ny-1), (Nz-1, Nz-1)).reshape(-1, 3),
                      coords((0, 0), (0, Ny-1), (0, Nz-1)).reshape(-1, 3),
                      coords((Nx-1, Nx-1), (0, Ny-1), (0, Nz-1)).reshape(-1, 3),
                      coords((0, Nx-1), (0, 0), (0, Nz-1)).reshape(-1, 3),
                      coords((0, Nx-1), (Ny-1, Ny-1), (0, Nz-1)).reshape(-1, 3)]


b_n = torch.tensor(bottom_values[:, 2], dtype=torch.float32)
r_p = torch.tensor(bottom_coords, dtype=torch.float32)

model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

pf_fields = []
pf_coords = []
for r_coords in top_lateral_coords:
    r_coords = torch.tensor(r_coords, dtype=torch.float32)
    pf_batch_size = np.prod(r_coords.shape[:-1]) // 10

    fields = []
    for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=409, num_workers=2),
                        desc='Potential Boundary'):
        r = r.to(device).requires_grad_(True)
        p_batch = model(r)
        b_p = -1 * torch.autograd.grad(p_batch, r, torch.ones_like(p_batch), retain_graph=True, create_graph=True)[0]
        fields += [b_p.clone().detach().cpu().numpy()]
    pf_fields += [np.concatenate(fields)]
    pf_coords += [r_coords.clone().detach().cpu().numpy()]

Potential Boundary: 100%|██████████| 11/11 [00:00<00:00, 102.34it/s]


In [98]:
pf_fields[0]

array([[-0.0797022 , -0.0439961 ,  0.05475029],
       [-0.08077709, -0.04406307,  0.05580638],
       [-0.08183653, -0.044113  ,  0.05689801],
       ...,
       [ 0.08183654,  0.044113  ,  0.05689799],
       [ 0.08077708,  0.04406307,  0.05580636],
       [ 0.0797022 ,  0.0439961 ,  0.05475028]], dtype=float32)

In [99]:
pf_coords[0]

array([[ 0.,  0., 63.],
       [ 0.,  1., 63.],
       [ 0.,  2., 63.],
       ...,
       [63., 61., 63.],
       [63., 62., 63.],
       [63., 63., 63.]], dtype=float32)

In [23]:
pf_batch_size = r_top_lateral.shape[0] // 100
b_n = torch.tensor(bz_bottom, dtype=torch.float32)
r_p = torch.tensor(r_bottom, dtype=torch.float32)
model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

r_coords= torch.tensor(r_top_lateral, dtype=torch.float32)

fields = []
for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
                    desc='Potential Boundary'):
    r = r.to(device).requires_grad_(True)
    p_batch = model(r)
    b_p = -1 * torch.autograd.grad(p_batch, r, torch.ones_like(p_batch), retain_graph=True, create_graph=True)[0]
    fields += [b_p.clone().detach().cpu().numpy()]
    # del r, p_batch, b_p

Potential Boundary: 100%|██████████| 101/101 [00:00<00:00, 183.87it/s]


In [35]:
b_p_a = np.concatenate(fields)
b_p_a

array([[-0.08263111, -0.04503536,  0.05397591],
       [-0.0797022 , -0.0439961 ,  0.05475029],
       [-0.07688085, -0.0429593 ,  0.05539294],
       ...,
       [ 0.08444643,  0.04591746,  0.05204187],
       [ 0.08147667,  0.04491767,  0.05295512],
       [ 0.07861352,  0.04391386,  0.05372883]], dtype=float32)

In [37]:
b_p.shape

(64, 64, 3, 3)

In [250]:
b_n = torch.tensor(bz_bottom, dtype=torch.float32)
r_p = torch.tensor(r_bottom, dtype=torch.float32)
model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)
r = r_coords[:1000].clone().detach().requires_grad_(True)
a = model(r)
a.shape

torch.Size([1000])

In [253]:
r

tensor([[ 0.,  0., 62.],
        [ 0.,  0., 63.],
        [ 0.,  0., 64.],
        ...,
        [ 5., 12., 63.],
        [ 5., 12., 64.],
        [ 5., 13., 62.]], requires_grad=True)

In [254]:
a

tensor([7.3860, 7.3316, 7.2765, 7.4311, 7.3756, 7.3195, 7.4763, 7.4197, 7.3625,
        7.5215, 7.4639, 7.4055, 7.5668, 7.5080, 7.4485, 7.6122, 7.5522, 7.4915,
        7.6576, 7.5963, 7.5344, 7.7029, 7.6404, 7.5772, 7.7483, 7.6843, 7.6199,
        7.7935, 7.7282, 7.6624, 7.8387, 7.7720, 7.7048, 7.8837, 7.8155, 7.7469,
        7.9285, 7.8589, 7.7888, 7.9731, 7.9019, 7.8304, 8.0175, 7.9447, 7.8717,
        8.0615, 7.9872, 7.9126, 8.1051, 8.0292, 7.9530, 8.1484, 8.0708, 7.9930,
        8.1911, 8.1119, 8.0325, 8.2334, 8.1524, 8.0714, 8.2750, 8.1923, 8.1096,
        8.3160, 8.2315, 8.1472, 8.3563, 8.2700, 8.1840, 8.3958, 8.3077, 8.2201,
        8.4344, 8.3446, 8.2552, 8.4721, 8.3805, 8.2894, 8.5088, 8.4154, 8.3227,
        8.5444, 8.4492, 8.3548, 8.5789, 8.4820, 8.3859, 8.6122, 8.5135, 8.4158,
        8.6441, 8.5438, 8.4444, 8.6748, 8.5727, 8.4718, 8.7040, 8.6002, 8.4978,
        8.7317, 8.6263, 8.5223, 8.7578, 8.6508, 8.5454, 8.7823, 8.6738, 8.5669,
        8.8050, 8.6951, 8.5869, 8.8261, 

In [259]:
torch.autograd.grad(a, r, torch.ones_like(a), create_graph=True)[0]

tensor([[ 0.0826,  0.0450, -0.0540],
        [ 0.0797,  0.0440, -0.0548],
        [ 0.0769,  0.0430, -0.0554],
        ...,
        [ 0.0929,  0.0471, -0.0893],
        [ 0.0889,  0.0453, -0.0888],
        [ 0.0980,  0.0484, -0.0916]], grad_fn=<GatherBackward>)

In [261]:
r_top_lateral[:1000]

array([[ 0,  0, 62],
       [ 0,  0, 63],
       [ 0,  0, 64],
       ...,
       [ 5, 12, 63],
       [ 5, 12, 64],
       [ 5, 13, 62]])

In [262]:
potential[:1000]

array([7.3859816, 7.3316073, 7.276525 , 7.4310718, 7.3756385, 7.319503 ,
       7.4762554, 7.419727 , 7.3625045, 7.521518 , 7.4638567, 7.405513 ,
       7.5668397, 7.5080094, 7.448504 , 7.6122   , 7.552158 , 7.4914613,
       7.657574 , 7.596285 , 7.5343566, 7.7029395, 7.6403575, 7.577161 ,
       7.748264 , 7.684349 , 7.619852 , 7.7935176, 7.728233 , 7.662396 ,
       7.8386664, 7.7719707, 7.70476  , 7.883676 , 7.8155293, 7.7469096,
       7.928504 , 7.858871 , 7.7888055, 7.9731107, 7.9019513, 7.830411 ,
       8.01745  , 7.944729 , 7.87168  , 8.061476 , 7.9871573, 7.912569 ,
       8.105138 , 8.029188 , 7.9530373, 8.14838  , 8.070773 , 7.9930305,
       8.19115  , 8.111855 , 8.0324955, 8.233386 , 8.152379 , 8.071388 ,
       8.275031 , 8.19229  , 8.109645 , 8.316018 , 8.231527 , 8.147215 ,
       8.356285 , 8.270028 , 8.184037 , 8.395764 , 8.30773  , 8.220054 ,
       8.434386 , 8.344568 , 8.255205 , 8.472084 , 8.380479 , 8.289426 ,
       8.508781 , 8.415396 , 8.32266  , 8.544411 , 

In [184]:
top_lateral_coords[0].shape

(64, 64, 3, 3)

In [185]:
p = potential[0:np.prod(r_top_lateral_shape[0])].reshape(r_top_lateral_shape[0])
p.shape

(64, 64, 3)

In [195]:
b_p = -1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
b_p.shape

(64, 64, 3, 3)

In [265]:
b_p.shape

(64, 64, 3, 3)

In [266]:
b_p[:, :, 1].shape

(64, 64, 3)

In [148]:
p = potential[0:np.prod(r_top_lateral_shape[0])].reshape(r_top_lateral_shape[0])

In [159]:
np.shape(np.array(np.gradient(p, axis=[0, 1, 2], edge_order=2)))

(3, 64, 64, 3)

False

In [163]:
np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)

array([[[[ 0.08266521,  0.04504395, -0.05401969],
         [ 0.07973576,  0.04400277, -0.05472827],
         [ 0.07691479,  0.04296613, -0.05543709]],

        [[ 0.08380508,  0.04513693, -0.05508184],
         [ 0.08081293,  0.04405975, -0.05578446],
         [ 0.07792568,  0.04298973, -0.05648708]],

        [[ 0.08493328,  0.04522324, -0.05618167],
         [ 0.08187222,  0.04410911, -0.05687547],
         [ 0.07892632,  0.04300499, -0.0575695 ]],

        ...,

        [[ 0.07440233, -0.02963066, -0.12062693],
         [ 0.07134247, -0.02882862, -0.11777258],
         [ 0.06842899, -0.02804899, -0.11491871]],

        [[ 0.07293701, -0.03146267, -0.11977053],
         [ 0.0699563 , -0.03057909, -0.11694145],
         [ 0.06711721, -0.02972364, -0.11411285]],

        [[ 0.07146692, -0.03326702, -0.11883163],
         [ 0.06856346, -0.03230476, -0.11603355],
         [ 0.06580114, -0.03137398, -0.11323547]]],


       [[[ 0.08329749,  0.04619765, -0.05705452],
         [ 0.0802691 ,

In [165]:
np.stack([np.array([1, 2, 3]), np.array([3, 4, 5])], -1)

array([[1, 3],
       [2, 4],
       [3, 5]])

In [162]:
np.gradient(p, axis=[0, 1, 2], edge_order=2)

[array([[[ 0.08266521,  0.07973576,  0.07691479],
         [ 0.08380508,  0.08081293,  0.07792568],
         [ 0.08493328,  0.08187222,  0.07892632],
         ...,
         [ 0.07440233,  0.07134247,  0.06842899],
         [ 0.07293701,  0.0699563 ,  0.06711721],
         [ 0.07146692,  0.06856346,  0.06580114]],
 
        [[ 0.08329749,  0.0802691 ,  0.07735658],
         [ 0.08445096,  0.0813539 ,  0.0783782 ],
         [ 0.08558846,  0.08242416,  0.0793829 ],
         ...,
         [ 0.07316923,  0.07011986,  0.06722403],
         [ 0.07167959,  0.06871176,  0.06589317],
         [ 0.07018232,  0.06729746,  0.0645566 ]],
 
        [[ 0.0838747 ,  0.08074951,  0.07775068],
         [ 0.08503914,  0.08184361,  0.07877779],
         [ 0.08618641,  0.08292103,  0.07978845],
         ...,
         [ 0.07185602,  0.06882429,  0.06594896],
         [ 0.07034254,  0.06739521,  0.06459904],
         [ 0.06882429,  0.06596279,  0.06324577]],
 
        ...,
 
        [[-0.06882572, -0.06596327

In [157]:
len(np.gradient(p, axis=[0, 1, 2], edge_order=2))

3

In [124]:
p

array([7.3859816, 7.3316073, 7.276525 , ..., 7.3859806, 7.3316073,
       7.276525 ], dtype=float32)

In [125]:
b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
n_points_top_lateral = [c.shape[:-1] for c in r_top_lateral]

idx = 0
fields = []
for n_points in n_points_top_lateral:
    p = potential[idx:idx + np.prod(n_points)].reshape(n_points)
    b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
    fields += [b]
    idx += np.prod(s)

fields = [fields[0][:, :, 1].reshape((-1, 3)),
            fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
            fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
coords = [coords[0][:, :, 1].reshape((-1, 3)),
            coords[1][:, 1, :].reshape((-1, 3)), coords[2][:, 1, :].reshape((-1, 3)),
            coords[3][1, :, :].reshape((-1, 3)), coords[4][1, :, :].reshape((-1, 3))]

In [72]:
r_top_lateral.shape

(61440, 3)

In [74]:
61440 // 100

614

In [71]:
bz_bottom.shape

(4096,)

In [60]:
bz_bottom[:, None]

array([[-2.10841989],
       [-2.19860577],
       [-2.28830561],
       ...,
       [-2.28830531],
       [-2.1986055 ],
       [-2.10841989]])

In [41]:
a = coords((7, 8), (7, 8), (9, 9))
b = coords((1, 2), (1, 1), (4, 4))

A = torch.tensor(a)
B = torch.tensor(b)

In [36]:
a

array([[7, 7, 9],
       [7, 8, 9],
       [8, 7, 9],
       [8, 8, 9]])

In [37]:
b

array([[1, 1, 4],
       [2, 1, 4]])

In [42]:
A

tensor([[7, 7, 9],
        [7, 8, 9],
        [8, 7, 9],
        [8, 8, 9]])

In [43]:
B

tensor([[1, 1, 4],
        [2, 1, 4]])

In [45]:
a[None, :] + b[:, None]

array([[[ 8,  8, 13],
        [ 8,  9, 13],
        [ 9,  8, 13],
        [ 9,  9, 13]],

       [[ 9,  8, 13],
        [ 9,  9, 13],
        [10,  8, 13],
        [10,  9, 13]]])

In [51]:
(A[None, :] + B[:, None])**2

tensor([[[ 64,  64, 169],
         [ 64,  81, 169],
         [ 81,  64, 169],
         [ 81,  81, 169]],

        [[ 81,  64, 169],
         [ 81,  81, 169],
         [100,  64, 169],
         [100,  81, 169]]])

In [64]:
aaa = torch.sqrt(torch.sum((A[None, :] + B[:, None])**2, -1))
aaa

tensor([[17.2337, 17.7200, 17.7200, 18.1934],
        [17.7200, 18.1934, 18.2483, 18.7083]])

In [68]:
k = torch.tensor([4, 4])

In [69]:
k[] / aaa

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1

In [58]:
torch.sum(aaa, 0)

tensor([34.9537, 35.9135, 35.9683, 36.9017])

In [47]:
torch.sum(A[None, :] + B[:, None], 0)

tensor([[17, 16, 26],
        [17, 18, 26],
        [19, 16, 26],
        [19, 18, 26]])

In [6]:
potential_numpy[:10]

array([7.38598259, 7.33160802, 7.27652588, 7.43107096, 7.37563895,
       7.31950322, 7.47625497, 7.41972848, 7.36250503, 7.5215176 ])

In [7]:
potential[:10]

array([7.385983 , 7.331608 , 7.276526 , 7.431071 , 7.37564  , 7.319504 ,
       7.4762526, 7.419729 , 7.362506 , 7.521517 ], dtype=float32)